### import Module

In [1]:
import torch
from einops import rearrange
from einops.layers.torch import Rearrange
import numpy as np

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 3090


### Make Patches

In [22]:
import torch.nn as nn
## input image batch 1000개의 Sample RGB 3개의 채널 height 244 X  width 244 크기이다. 
x = torch.randn(1000, 3, 224, 224)

## Image를 이미지의 크기/패치의 크기의 패치로 쪼개고 Flatten 시킨다. 
p = 16 # 논문에서 16이 계산 비용대비 효율적이라 소개한다. 

## 이때, p_h(p_w): 패치의 크기 and n_h(n_w): 패치의 크기 나눴을 때 가로 수 
patches = rearrange(x, 'b c (p_h n_h) (p_w n_w) -> b (n_h n_w) (p_h p_w c)', p_h=p, p_w=p)


########### 결과
# x : torch.Size([1000, 3, 224, 224])
# patches : torch.Size([1000, 196, 768])

In [25]:
from torchsummary import summary
import pandas as pd
import math 

in_channel = 3
patch_size = 16
embedding = pow(patch_size, 2) * in_channel

patch_layer = nn.Sequential(
    nn.Conv2d(in_channel, embedding, kernel_size= patch_size, stride= patch_size),
    Rearrange('b e h w -> b (h w) e')
).cuda()
##  input을 여러 개 받는 다면 summary(model, [(1, 16, 16), (1, 28, 28)]) 이렇게 리스트로 담아주자
summary(patch_layer,  x.shape[1:], device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
Total params: 590,592
Trainable params: 590,592
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 2.30
Params size (MB): 2.25
Estimated Total Size (MB): 5.12
----------------------------------------------------------------


### Add Class token and Position Encoding 

In [ ]:
# (여담이지만, 개인적으로 class token 없어도 학습이 잘 이루어지는걸 확인했는데, 모델의 마지막 layer인 linear layer에서 모델의 모든 feature들이 계산되는걸 보면 굳이 없어도 되지 않은가 싶었다.)
# patch 당 Class token이 붙는다. (왜 사진 당 클래스 토큰을 안붙였지? 내가 아는 클래스가 아닌 것인가...??)


tensor([[-0.0645,  0.0319, -0.0629,  ...,  0.0329, -0.0006,  0.0061],
        [ 0.1477, -0.0162,  0.0389,  ...,  0.0314, -0.0117,  0.0088],
        [-0.0173, -0.0504, -0.0197,  ...,  0.0237, -0.0635, -0.1069],
        ...,
        [-0.0596, -0.0419, -0.0436,  ..., -0.0769, -0.0583, -0.0541],
        [ 0.0225,  0.0889, -0.0031,  ...,  0.0488, -0.0606,  0.0227],
        [ 0.0221,  0.1643,  0.0616,  ...,  0.0945, -0.1091,  0.0279]])


In [6]:
*[1,2,3]

SyntaxError: can't use starred expression here (386627056.py, line 1)